# Find Pairwise Interactions
This notebook demonstrates how to calculate pairwise intra- and inter-molecular interactions at specified levels of granularity within biological assemblies and asymmetric units.

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.utils import ColumnarStructure
from mmtfPyspark.interactions import InteractionExtractorPd

### Start a Spark Session

In [2]:
spark = SparkSession.builder.appName("Interactions").getOrCreate()

## Define Interaction Partners
Interactions are defined by specifing two subsets of atoms, named **query** and **target**. Once defined, interactions can calculated between these two subsets.

### Use Pandas Dataframes to Create Subsets
The InteractionExtractorPd internally uses Pandas dataframe queries to create query and target atom sets. Any of the Pandas column names below can be used to create subsets.

Example of a structure represented in a Pandas dataframe.

In [3]:
structures = mmtfReader.download_mmtf_files(["1OHR"]).cache()

# get first structure from Spark RDD (keys = PDB IDs, value = mmtf structures)
first_structure = structures.values().first()

# convert to a Pandas dataframe
#df = ColumnarStructure(first_structure).to_pandas()
df = ColumnarStructure(first_structure)
#df.head(5)

In [ ]:
df.to_pandas()

### Create a subset of atoms using boolean expressions
The following query creates a subset of ligand (non-polymer) atoms that are not water (HOH) or heavy water (DOD).

In [ ]:
query = "not polymer and (group_name not in ['HOH','DOD'])"
df_lig = df.query(query)
df_lig.head(5)

## Calculate Interactions
 The following boolean expressions specify two subsets: ligands (query) and polymer groups (target). In this example, interactions within a distance cutoff of 4 &#197; are calculated.

In [ ]:
query = "not polymer and (group_name not in ['HOH','DOD'])"
target = "polymer"
distance_cutoff = 4.0

# the result is a Spark dataframe
interactions = InteractionExtractorPd.get_interactions(structures, distance_cutoff,
                                                       query, target)

# get the first 5 rows of the Spark dataframe and display it as a Pandas dataframe
interactions.limit(5).toPandas()

## Calculate all interactions
If query and target are not specified, all interactions are calculated. By default, intermolecular interactions are calculated.

In [ ]:
interactions = InteractionExtractorPd.get_interactions(structures, distance_cutoff)
interactions.limit(5).toPandas()

## Aggregate Interactions at Different Levels of Granularity
Pairwise interactions can be listed at different levels of granularity by setting the **level**:
* **level='coord'**: pairwise atom interactions, distances, and coordinates
* **level='atom'**:  pairwise atom interactions and distances
* **level='group'**: pairwise atom interactions aggregated at the group (residue) level (default)
* **level='chain'**: pairwise atom interactions aggregated at the chain level

The next example lists the interactions at the **coord** level, the level of highest granularity. You need to scroll in the dataframe to see all columns.

In [ ]:
interactions = InteractionExtractorPd.get_interactions(structures, distance_cutoff,
                                                         query, target, level='coord')
interactions.limit(5).toPandas()

## Calculate Inter- vs Intra-molecular Interactions
Inter- and intra-molecular interactions can be calculated by explicitly setting the **inter** and **intra** flags.
* **inter=True** (default)
* **intra=False** (default)

### Find intermolecular salt-bridges
This example uses the default settings, i.e., finds intramolecular salt-bridges.

In [ ]:
query = "polymer and (group_name in ['ASP', 'GLU']) and (atom_name in ['OD1', 'OD2', 'OE1', 'OE2'])"
target = "polymer and (group_name in ['ARG', 'LYS', 'HIS']) and (atom_name in ['NH1', 'NH2', 'NZ', 'ND1', 'NE2'])"
distance_cutoff = 3.5
    
interactions = InteractionExtractorPd.get_interactions(structures, distance_cutoff,
                                                         query, target, level='atom')
interactions.limit(5).toPandas()

### Find intramolecular hydrogen bonds
In this example, the inter and intra flags have been set to find intramolecular hydrogen bonds.

In [ ]:
query = "polymer and element in ['N','O']"
target = "polymer and element in ['N','O']"
distance_cutoff = 3.5

interactions = InteractionExtractorPd.get_interactions(structures, distance_cutoff,
                                                       query, target, 
                                                       inter=False, intra=True,
                                                       level='atom')
interactions.limit(5).toPandas()

## Calculate Interaction in the Biological Assembly vs. Asymmetric Unit

In [ ]:
structures = mmtfReader.download_mmtf_files(["1STP"]).cache()

By default, interactions in the first biological assembly are calculated. The **bio** parameter specifies the biological assembly number. Most PDB structure have only one biological assembly (bio=1), a few have more than one.
* **bio=1** use first biological assembly (default)
* **bio=2** use second biological assembly
* **bio=None** use the asymmetric unit

In [ ]:
query = "not polymer and (group_name not in ['HOH','DOD'])"
target = "polymer"
distance_cutoff = 4.0

# The asymmetric unit is a monomer (1 ligand, 1 protein chain)
interactions = InteractionExtractorPd.get_interactions(structures, distance_cutoff,
                                                       query, target, bio=None)
print("Ligand interactions in asymmetric unit (monomer)        :", interactions.count())

# The first biological assembly is a tetramer (4 ligands, 4 protein chain)
interactions = InteractionExtractorPd.get_interactions(structures, distance_cutoff,
                                                       query, target, bio=1)
print("Ligand interactions in 1st bio assembly (tetramer)      :", interactions.count())

# There is no second biological assembly, in that case zero interactions are returned
interactions = InteractionExtractorPd.get_interactions(structures, distance_cutoff,
                                                       query, target, bio=2)
print("Ligand interactions in 2st bio assembly (does not exist):", interactions.count())

The 1st biological unit contains 68 - 4x16 = 4 additional interactions not found in the asymmetric unit.

## Stop Spark!

In [ ]:
spark.stop()